In [ ]:
!pip install transformers

In [ ]:
import torch
import transformers as ppb
import pandas as pd
import numpy as np

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'cse6242project-417410'
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!curl https://sdk.cloud.google.com | bash

In [ ]:
bucket_name = 'team054'
# Download the file from the given Google Cloud Storage bucket.
!gsutil cp gs://team054/books_enriched.csv books_enriched.csv

Copying gs://team054/books_enriched.csv...
/ [1 files][ 13.0 MiB/ 13.0 MiB]                                                
Operation completed over 1 objects/13.0 MiB.                                     


In [ ]:
import pandas as pd
from ast import literal_eval
#books_df = pd.read_csv('/content/books_enriched.csv')#---if uploading through the bucket way,then uncomment this

books_df = pd.read_csv('/content/drive/MyDrive/CSE6242_project/books_enriched.csv')

#books_df = pd.read_csv('/content/drive/MyDrive/CSE6242_project/combined_processed_files.txt')  --- Terry's file


books_df = books_df[:1000]

In [ ]:
books_df['short_description'] = books_df.description.str.slice(0,512)

In [ ]:


df = books_df.short_description

In [ ]:
tokenized = df.apply((lambda x: tokenizer.encode(str(x), add_special_tokens=True)))

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

# Embed Text

In [ ]:
#device = torch.device('cuda')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_ids = torch.tensor(padded).to(device)
attention_mask = torch.tensor(attention_mask).to(device)
model = model.to(device)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
embeddings = last_hidden_states[0].cpu().data[:,0,:].numpy()

In [ ]:
df.shape, embeddings.shape

((1000,), (1000, 768))

# Save Embeddings

In [ ]:
embeddings_pd_df = pd.DataFrame({'book_id': range(df.shape[0])})
embeddings_pd_df['title'] = books_df['title']
embeddings_pd_df['embedding'] = embeddings.tolist()

In [ ]:

embeddings_pd_df['embedding'] = embeddings_pd_df['embedding'].apply(lambda x: np.array(x))


In [ ]:
embeddings_pd_json= embeddings_pd_df.to_json()

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
df.to_json('/content/drive/MyDrive/CSE6242_project/embeddings_pd_json.json')

In [ ]:
#!gsutil cp embeddings_pd_txt gs://team054

In [ ]:
#import datalab.storage as gcs
#gcs.Bucket('team054').item('to/embeddings_pd_json.json').write_to(simple_dataframe.to_json(),'text/json')

In [ ]:
#from google.cloud import storage

#def write_to_cloud(buffer):
 #   client = storage.Client()
  #  bucket = client.get_bucket('team054')
  #  blob = bucket.blob('gs://team054/embeddings_pd_json.json')
 #   blob.upload_from_file(buffer)

In [ ]:
#from google.cloud import storage

#def create_newfolder(bucket_name, destination_folder_name):
 #   storage_client = storage.Client()
 #   bucket = storage_client.get_bucket('team054')
  #  blob = bucket.blob('output')

  #  blob.upload_from_file('')

   # print('Created {} .'.format(destination_folder_name))

In [ ]:
embeddings_pd_df.head()

,book_id,title,embedding
0,0,"The Hunger Games (The Hunger Games, #1)","[0.11946984380483627, 0.057541050016880035, 0...."
1,1,Harry Potter and the Sorcerer's Stone (Harry P...,"[-0.3747820258140564, -0.2721218168735504, 0.1..."
2,2,"Twilight (Twilight, #1)","[0.12502743303775787, -0.7245563268661499, -0...."
3,3,To Kill a Mockingbird,"[-0.4378758668899536, -0.29153937101364136, -0..."
4,4,The Great Gatsby,"[-0.4059891104698181, -0.47040677070617676, -0..."


# Train KNN

In [ ]:
from sklearn.neighbors import NearestNeighbors

X = np.array(embeddings_pd_df['embedding'].values.tolist())
k = 5
nn_model = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(X)


# Query KNN

In [ ]:
query = np.array([X[i] + np.random.normal(loc=0.0, scale=1.0, size=X[0].shape) for i in range(2)])

In [ ]:
distances, indices = nn_model.kneighbors(query)

In [ ]:
i = 0
for book_rec in indices:
    i += 1
    j = 0
    print(f"Top {k} recommendations for query {i}:")
    for index in book_rec:
        j += 1
        print(f"\t{j}. {embeddings_pd_df.iloc[index]['title']}")

Top 5 recommendations for query 1:
	1. The Hunger Games (The Hunger Games, #1)
	2. A Court of Thorns and Roses (A Court of Thorns and Roses, #1)
	3. Inkheart (Inkworld, #1)
	4. The Way of Kings (The Stormlight Archive, #1)
	5. A Dance with Dragons (A Song of Ice and Fire, #5)
Top 5 recommendations for query 2:
	1. Harry Potter and the Sorcerer's Stone (Harry Potter, #1)
	2. Digital Fortress
	3. Fruits Basket, Vol. 1
	4. Ender's Game (Ender's Saga, #1)
	5. Club Dead (Sookie Stackhouse, #3)
